In [0]:
# https://deeplearningcourses.com/c/advanced-computer-vision
# https://www.udemy.com/advanced-computer-vision
from __future__ import print_function, division
from builtins import range, input
# Note: you may need to update your version of future
# sudo pip install -U future

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt







# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=(100,100,3), weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(120, activation='softmax')(x)


# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='rmsprop',
  metrics=['accuracy']
)





Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0   

In [0]:
! pip install -q kaggle

In [0]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"gokulnath31","key":"8bbd6742ff45eb5ce3f23eecc64519e9"}'}

In [0]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

ERROR! Session/line number was not unique in database. History logging moved to new session 59


In [0]:
!kaggle datasets download -d moltean/fruits

 98% 723M/734M [00:04<00:00, 189MB/s]
100% 734M/734M [00:04<00:00, 155MB/s]


In [0]:
!ls

fruits	fruits.zip  kaggle.json  sample_data


In [0]:
from shutil import copyfile
import os
import zipfile
import random

In [0]:
local_zip = '/content/fruits.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [0]:
import os
path = '/tmp/fruits-360_dataset/fruits-360/Training'
filenames = os.listdir(path)
for filename in filenames:
  os.rename(os.path.join(path, filename), os.path.join(path, filename.replace(' ', '-')))

In [0]:
os.mkdir("/tmp/fruits")
os.mkdir("/tmp/fruits/train")
os.mkdir("/tmp/fruits/validate")

In [0]:
classes=[str(x) for x in os.listdir("/tmp/fruits-360_dataset/fruits-360/Training/")]
sets=["train", "validate"]
p="/tmp/fruits/"
for s in sets:
  for c in classes:
    os.mkdir(str(p)+str(s)+"/"+str(c))

In [0]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[:testing_length]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)






In [0]:
t="/tmp/fruits/train/"
v="/tmp/fruits/validate/"
src="/tmp/fruits-360_dataset/fruits-360/Training/"


split_size = .8


for c in classes:
  SOURCE=src+c+"/"
  TRAIN=t+c+"/"
  VALIDATE=v+c+"/"
  print(SOURCE,TRAIN,VALIDATE)

/tmp/fruits-360_dataset/fruits-360/Training/Cherry-1/ /tmp/fruits/train/Cherry-1/ /tmp/fruits/validate/Cherry-1/
/tmp/fruits-360_dataset/fruits-360/Training/Pomelo-Sweetie/ /tmp/fruits/train/Pomelo-Sweetie/ /tmp/fruits/validate/Pomelo-Sweetie/
/tmp/fruits-360_dataset/fruits-360/Training/Apple-Red-2/ /tmp/fruits/train/Apple-Red-2/ /tmp/fruits/validate/Apple-Red-2/
/tmp/fruits-360_dataset/fruits-360/Training/Hazelnut/ /tmp/fruits/train/Hazelnut/ /tmp/fruits/validate/Hazelnut/
/tmp/fruits-360_dataset/fruits-360/Training/Eggplant/ /tmp/fruits/train/Eggplant/ /tmp/fruits/validate/Eggplant/
/tmp/fruits-360_dataset/fruits-360/Training/Grape-White-3/ /tmp/fruits/train/Grape-White-3/ /tmp/fruits/validate/Grape-White-3/
/tmp/fruits-360_dataset/fruits-360/Training/Apricot/ /tmp/fruits/train/Apricot/ /tmp/fruits/validate/Apricot/
/tmp/fruits-360_dataset/fruits-360/Training/Grape-White/ /tmp/fruits/train/Grape-White/ /tmp/fruits/validate/Grape-White/
/tmp/fruits-360_dataset/fruits-360/Training/Appl

In [0]:
t="/tmp/fruits/train/"
v="/tmp/fruits/validate/"
src="/tmp/fruits-360_dataset/fruits-360/Training/"


split_size = .8


for c in classes:
  SOURCE=src+c+"/"
  TRAIN=t+c+"/"
  VALIDATE=v+c+"/"
  split_data(SOURCE, TRAIN, VALIDATE, split_size)








In [0]:
TRAINING_DIR = "/tmp/fruits/train/"
# Experiment with your own parameters here to really try to drive it to 99.9% accuracy or better
train_datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    class_mode='categorical',
                                                    target_size=(100, 100))

VALIDATION_DIR = "/tmp/fruits/validate/"
# Experiment with your own parameters here to really try to drive it to 99.9% accuracy or better
validation_datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              class_mode='categorical',
                                                              target_size=(100, 100))

Found 48360 images belonging to 120 classes.
Found 12138 images belonging to 120 classes.


In [0]:
print(len(classes))

120


In [0]:
history = model.fit_generator(train_generator,
                              epochs=25,
                              verbose=1,
                              validation_data=validation_generator)

Epoch 1/25
  12703/Unknown - 40176s 3s/step - loss: 0.4026 - accuracy: 0.8869